**第10章**

In [1]:
from myABBR import * ;import myUtil as mu ; np.set_printoptions(precision=6) 
# 初期値  bS:buySell: Protection.Buyer=0
tradeDT = jDT(2022,9,19) ; setEvDT(tradeDT)
matDT,           bS,   ntlAMT,   rfRT,  cdsQT,  npv0, hzdRT                 =\
jDT(2023,9,20),  0,  10_000_000, 0.10, 1.30/100, 0,   0;hzdRT=cdsQT/(1-rcvRTj)

# DF,hazard rateカーブ
dsCvOBJ, dsCvHDL = mu.ffTSH(tradeDT, rfRT, dcA365)
hzCvOBJ = ql.FlatHazardRate(tradeDT,mu.sqHDL(hzdRT), dcA365)
hzCvHDL = ql.DefaultProbabilityTermStructureHandle(hzCvOBJ)
print(f'(満期日) DF:{dsCvOBJ.discount(matDT):.6f}',            end=', ') 
print(  f'生存確率Q:{hzCvOBJ.survivalProbability(matDT):.6%}', end=', ') 
print(  f'ﾊｻﾞｰﾄﾞﾚｰﾄ:{hzCvOBJ.hazardRate(matDT):.6%}'                   ) 

(満期日) DF:0.904590, 生存確率Q:98.014497%, ﾊｻﾞｰﾄﾞﾚｰﾄ:2.000000%


In [2]:
# cds NPV計算
cdsSCD = ql.Schedule(tradeDT+1, matDT, pdFreqQ, calWK, FLLW, 
                                                unADJ, dtGENf, EoMf)
cdsOBJ = ql.CreditDefaultSwap(bS, ntlAMT, cpn100, cdsSCD, FLLW, dcA360)
midENG = ql.MidPointCdsEngine(hzCvHDL, rcvRTj, dsCvHDL)
cdsOBJ.setPricingEngine(midENG)

# cpnLEGの違いは3日分のUpfront金額か？？？
print(f'tradeDate     :{cdsOBJ.tradeDate().ISO():}')
print(f'UpfrontDate   :{cdsOBJ.upfrontPayment().date().ISO():}')
print('----------------')
print(f'couponLegNPV  :{cdsOBJ.couponLegNPV():,.2f}')
print(f'defaultLegNPV :{cdsOBJ.defaultLegNPV():,.2f}')
print(f'net Two Leg   :{cdsOBJ.couponLegNPV()+cdsOBJ.defaultLegNPV():,.2f}')
print('----------------')
print(f'NPV           :{cdsOBJ.NPV():,.2f}')
print(f'fairSpread    :{cdsOBJ.fairSpread():.6%}')
print(f'hzdRT (inOBJ) :\
{cdsOBJ.impliedHazardRate(cdsOBJ.NPV(),dsCvHDL,dcA365,rcvRTj):.6%}')
print(f'accrual amount:  {cdsOBJ.accrualRebate().amount():,.10f}')

tradeDate     :2022-09-19
UpfrontDate   :2022-09-22
----------------
couponLegNPV  :-94,323.10
defaultLegNPV :122,462.50
net Two Leg   :28,139.40
----------------
NPV           :28,139.40
fairSpread    :1.298330%
hzdRT (inOBJ) :2.000000%
accrual amount:  0.0000000000
